This notebook illustrates a situation where some perturbed genes are not presented in the gene embedding matrix. \
There are two possibilities causes the unpresented genes:
- The provider of the gene embedding matrix did not include that gene. In this case, one can manually collect the text description of the gene of interest, and retrieve its corresponding embedding from LLM using, for example, OpenAI's api.
- Perturbed genes are not found in the embedding matrix due to different symbols for an actually same gene.

In this notebook, we illsutrate the second case, on the gene embeddings provided by scELMo together with Adamson dataset. We hope this notebook to serve as an reminder that users should always check the name matches between gene embedding matrix and gene expression data.

In [1]:
import pickle
import anndata as ad
import pandas as pd
import numpy as np
from scouter import ScouterData

In [2]:
data_path = '/Users/pancake/Downloads/Perturb/Gears/adamson/Gears_data/adamson/perturb_processed.h5ad'
embd_path = '/Users/pancake/Downloads/Perturb/scOracle/GeneEmb/scELMO_emb/ensem_emb_gpt3.5all_new.pickle'

In [3]:
# Load the processed scRNA-seq dataset as Anndata
adata = ad.read_h5ad(data_path)

# Load the gene embedding as the dataframe, and rename its gene alias to match the Anndata
with open(embd_path, 'rb') as f:
    embd = pd.DataFrame(pickle.load(f)).T
ctrl_row = pd.DataFrame([np.zeros(embd.shape[1])], columns=embd.columns, index=['ctrl'])
embd = pd.concat([ctrl_row, embd])

In [4]:
pertdata = ScouterData(adata, embd, 'condition', 'gene_name')
pertdata.setup_ad('embd_index')

10 perturbed genes are not found in the gene embedding matrix: 
['AARS' 'CARS' 'DARS' 'HARS' 'MARS' 'QARS' 'SARS' 'SLMO2' 'SRPR' 'TARS']. 
Hence they are deleted. Please check if this is because of different gene synonyms. 
Please check if the deletion of following conditions are correct: 
['AARS+ctrl', 'CARS+ctrl', 'DARS+ctrl', 'HARS+ctrl', 'MARS+ctrl', 'QARS+ctrl', 'SARS+ctrl', 'SLMO2+ctrl', 'SRPR+ctrl', 'TARS+ctrl']


10 out of 86 perturbed genes are not found in the scELMo's embedding matrix. **But are they real unmatches?** \
Let's find out their corresponding EnsemblIDs, which are supposed to be unique and non-ambiguous

In [5]:
unmatch_names = pertdata.unmatched_genes
gene_df = adata.var.copy()
unmatch_df = gene_df[gene_df.gene_name.isin(unmatch_names)]
unmatch_df

,gene_name
ENSG00000031698,SARS
ENSG00000115866,DARS
ENSG00000172053,QARS
ENSG00000113407,TARS
ENSG00000170445,HARS
ENSG00000110619,CARS
ENSG00000182934,SRPR
ENSG00000166986,MARS
ENSG00000090861,AARS
ENSG00000101166,SLMO2


Given the EnsemblIDs, let's find out what their gene symbols are on Ensembl, using a python package `mygene`

In [6]:
import mygene
ids = list(unmatch_df.index)
name_adata = list(unmatch_df.gene_name)

embd_adata = {}
for e_id, name_adata in zip(ids, name_adata):
    mg = mygene.MyGeneInfo()
    result = mg.query(e_id, fields="symbol")
    gene_symbol = result["hits"][0]["symbol"] if result["total"] > 0 else None
    is_in = gene_symbol in embd.index
    print(f"{e_id}'s symbol on Ensembl is {gene_symbol}, and it is{'' if is_in else ' Not'} in the gene embedding matrix")
    embd_adata[gene_symbol] = name_adata

ENSG00000031698's symbol on Ensembl is SARS1, and it is in the gene embedding matrix
ENSG00000115866's symbol on Ensembl is DARS1, and it is in the gene embedding matrix
ENSG00000172053's symbol on Ensembl is QARS1, and it is in the gene embedding matrix
ENSG00000113407's symbol on Ensembl is TARS1, and it is in the gene embedding matrix
ENSG00000170445's symbol on Ensembl is HARS1, and it is in the gene embedding matrix
ENSG00000110619's symbol on Ensembl is CARS1, and it is in the gene embedding matrix
ENSG00000182934's symbol on Ensembl is SRPRA, and it is in the gene embedding matrix
ENSG00000166986's symbol on Ensembl is MARS1, and it is in the gene embedding matrix
ENSG00000090861's symbol on Ensembl is AARS1, and it is in the gene embedding matrix
ENSG00000101166's symbol on Ensembl is PRELID3B, and it is in the gene embedding matrix


**We rename the gene symbols in the embedding matrix to accomodate the gene expression adata, and now all genes are matched!**

In [7]:
embd.rename(index=embd_adata, inplace=True)
pertdata = ScouterData(adata, embd, 'condition', 'gene_name')
pertdata.setup_ad('embd_index')

All 87 perturbed genes are found in the gene embedding matrix!
